# Load Zillow Home Value Index

In [1]:
import pandas as pd
import numpy as np

housing_price_data = pd.read_csv("zillow_home_value_index_ignored.csv")
housing_price_data

,RegionID,SizeRank,RegionName,RegionType,StateName,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,2000-01-31,...,2021-05-31,2021-06-30,2021-07-31,2021-08-31,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28
0,3101,0,Los Angeles County,County,CA,CA,Los Angeles-Long Beach-Anaheim,6,37,216775.0,...,748570.0,766377.0,786335.0,798357.0,807005.0,809539.0,814096.0,819459.0,824634.0,833061.0
1,139,1,Cook County,County,IL,IL,Chicago-Naperville-Elgin,17,31,177178.0,...,278651.0,281701.0,285275.0,289056.0,291669.0,294182.0,297109.0,299865.0,302464.0,304239.0
2,1090,2,Harris County,County,TX,TX,Houston-The Woodlands-Sugar Land,48,201,114933.0,...,227793.0,232679.0,238034.0,243803.0,248213.0,251998.0,254585.0,257648.0,260840.0,264465.0
3,2402,3,Maricopa County,County,AZ,AZ,Phoenix-Mesa-Scottsdale,4,13,144005.0,...,374747.0,386230.0,399576.0,409614.0,417871.0,425408.0,431538.0,437309.0,443086.0,451812.0
4,2841,4,San Diego County,County,CA,CA,San Diego-Carlsbad,6,73,229369.0,...,742452.0,766854.0,790916.0,805688.0,815494.0,820274.0,827876.0,839310.0,859927.0,880414.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2820,425,3101,Camas County,County,ID,ID,Hailey,16,25,NaN,...,180576.0,183120.0,185071.0,187731.0,189472.0,191781.0,192706.0,198573.0,203367.0,209098.0
2821,1648,3102,Daggett County,County,UT,UT,NaN,49,9,NaN,...,253602.0,259555.0,263261.0,267800.0,271934.0,276307.0,279040.0,285972.0,291338.0,295519.0
2822,2345,3112,Hinsdale County,County,CO,CO,NaN,8,53,NaN,...,374095.0,378797.0,381258.0,385808.0,389654.0,393205.0,393935.0,399613.0,406698.0,413738.0
2823,1933,3122,Mineral County,County,CO,CO,NaN,8,79,NaN,...,318784.0,324480.0,330499.0,336036.0,341149.0,344634.0,345718.0,347980.0,349891.0,351719.0


# Make the new dataframe

In [2]:
stats = []

for index, row in housing_price_data.iterrows():
    name = row["RegionName"]
    state_code = row["StateCodeFIPS"]
    region_code = row["MunicipalCodeFIPS"]
    GISJOIN = f"G{str(state_code).zfill(2)}{str(region_code).zfill(4)}0"
    # housing prices peaked in early 2006 https://en.wikipedia.org/wiki/United_States_housing_bubble#:~:text=Housing%20prices%20peaked%20in%20early,price%20drop%20in%20its%20history.
    # and began recovering by 2012
    # so lets get the prices from the start of 2006 to the end of 2012
    column_labels = list(housing_price_data)
    start_of_2006_index = column_labels.index("2006-01-31") 
    end_of_2012_index = 12 * 7 - 1 + start_of_2006_index
    relevant_months = [column_labels[indx] for indx in range(start_of_2006_index, end_of_2012_index + 1)]
    prices = row[relevant_months].to_numpy()
    
    # we can assume the peak was between 2006-2008, so lets get the max from that range as our max
    highest_index = prices[:24].argmax()
    highest_price = prices[highest_index]
    
    # skip if we dont have a highest price 
    if np.isnan(highest_price):
        continue
        
    # we can assume the lowest price is simply the lowest price in our range
    lowest_index = prices.argmin()
    lowest_price = prices[lowest_index]
    
    if lowest_index < highest_index:
        continue
        
    # the "crash level" is simply the difference divided by the highest price
    crash_amount = (highest_price - lowest_price) / highest_price
    
    stats.append([name, GISJOIN, highest_price, lowest_price, crash_amount])
    
crash_amount_df = pd.DataFrame(stats, columns=["county", "GISJOIN", "highest_price", "lowest_price", "crash_amount"])

In [3]:
crash_amount_df.to_csv('crash_amounts.csv')